In [3]:
!pip install kaggle >> /dev/null

In [ ]:
!kaggle datasets download -d agrigorev/clothing-dataset-full

In [1]:
from fastcore.xtras import Path
zip_path = Path("../clothing-dataset-full.zip")
zip_path.exists()

In [2]:
import zipfile
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("../data")

In [3]:
data_path = Path("../data")
data_path.ls()

In [4]:
from fastai.vision.all import *

In [5]:
(data_path/"images_compressed").ls()[:3]

In [6]:
im_path = (data_path/"images_compressed").ls()[0]
im = Image.open(im_path)
im

In [7]:
im.shape

In [8]:
im_path = (data_path/"images_original").ls()[0]
im = Image.open(im_path);

In [9]:
im.shape

In [12]:
bad_imgs = []
for im in (data_path/"images_compressed").ls():
    try:
        _ = Image.open(im)
    except:
        bad_imgs.append(im)
        im.unlink()

In [13]:
len(bad_imgs)

In [14]:
df = pd.read_csv(data_path/'images.csv')
df.head()

In [15]:
len(df[df["label"] == "Not sure"]), len(df)

In [16]:
len(df[~(df["label"] == "Not sure")])

In [17]:
clean_df = df[~(df["label"] == "Not sure")]
clean_df.head()

In [24]:
for img in bad_imgs:
    clean_df = clean_df[clean_df["image"] != img.stem]

In [25]:
len(clean_df)

In [26]:
clean_df["label"].unique(), len(clean_df["label"].unique())

In [27]:
blocks = (ImageBlock, CategoryBlock)

In [28]:
get_x = ColReader("image", pref=(data_path/"images_compressed"), suff=".jpg")
get_y = ColReader("label")

In [29]:
item_tfms = [Resize(224)]
batch_tfms = [*aug_transforms(), Normalize.from_stats(*imagenet_stats)]

In [30]:
dblock = DataBlock(
    blocks=blocks,
    get_x=get_x,
    get_y=get_y,
    item_tfms=item_tfms,
    batch_tfms=batch_tfms
)

In [31]:
dls = dblock.dataloaders(clean_df)

In [32]:
dls.show_batch()

In [33]:
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(5)